In [41]:
import sys
sys.path.append('/Users/samrelins/Documents/BIHR/transport_proj/src')

from journey_time_helpers import *

In [42]:
data_dir = '/Users/samrelins/Documents/BIHR/transport_proj/data'
raw_jt_data = pd.read_csv(data_dir + "/jt_data_joined.csv")
raw_jt_data

,LSOA_code,Region,LA_Code,LA_Name,Empl_pop,100EmpPTt,100EmpPT15n,100EmpPT30n,100EmpPT45n,100EmpPT60n,...,TownWalk30n,TownWalk45n,TownWalk60n,TownWalk15pct,TownWalk30pct,TownWalk45pct,TownWalk60pct,LSOA_name,imd_rank,imd_decile
0,E01000001,London,E09000001,City of London,1189.0,20.085417,0.000000,10.0,10.0,10.0,...,0.657534,4.000000,8.710046,0.000000,65.753424,100.0,100.0,City of London 001A,29199,9
1,E01000002,London,E09000001,City of London,1131.0,18.172184,0.000000,10.0,10.0,10.0,...,1.000000,3.390361,8.219277,0.000000,100.000000,100.0,100.0,City of London 001B,30379,10
2,E01000003,London,E09000001,City of London,1334.0,19.404723,0.000000,10.0,10.0,10.0,...,1.000000,4.000000,9.669523,0.000000,100.000000,100.0,100.0,City of London 001C,14915,5
3,E01000005,London,E09000001,City of London,1416.0,10.701377,3.478814,10.0,10.0,10.0,...,0.695931,2.569593,9.000000,0.000000,43.040686,100.0,100.0,City of London 001E,8678,3
4,E01000006,London,E09000002,Barking and Dagenham,1489.0,2.997475,8.000000,10.0,10.0,10.0,...,1.000000,3.000000,6.558011,100.000000,100.000000,100.0,100.0,Barking and Dagenham 016A,14486,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32839,E01033764,North West,E08000012,Liverpool,2176.0,8.317551,4.420496,10.0,10.0,10.0,...,1.934300,3.374573,4.906997,10.238908,100.000000,100.0,100.0,Liverpool 022E,116,1
32840,E01033765,North West,E08000012,Liverpool,1105.0,3.227385,4.254299,10.0,10.0,10.0,...,1.000000,1.746586,3.025797,0.000000,100.000000,100.0,100.0,Liverpool 061D,945,1
32841,E01033766,North West,E08000012,Liverpool,803.0,3.174458,8.764633,10.0,10.0,10.0,...,2.000000,3.000000,5.000000,100.000000,100.000000,100.0,100.0,Liverpool 042G,12842,4
32842,E01033767,North West,E08000012,Liverpool,765.0,5.809595,5.666667,10.0,10.0,10.0,...,0.000000,2.000000,2.528470,0.000000,0.000000,100.0,100.0,Liverpool 050J,422,1


In [43]:
pop_cols = [col for col in raw_jt_data.columns if "pop" in col]
lsoa_level_cols = ['LSOA_code', 'Region', 'LA_Code', 
                   'LA_Name', "imd_rank", "imd_decile", "LSOA_name"]
jt_cols = [col for col in raw_jt_data.columns 
           if col not in pop_cols and col not in lsoa_level_cols]

# melt all journey time columns into dft_variable_name / value columns
lsoa_journey_times = raw_jt_data.melt(id_vars=["LSOA_code"],  
                                      value_vars=jt_cols, 
                                      var_name="dft_variable_name")

# parse out service names from dft_variable_name
service_names = {
    "100Emp": "Employment Sites - Small",
    "500Emp": "Employment Sites - Medium",
    "5000Emp": "Employment Sites - Large",
    "Empl": "Employment Sites",
    "PS": "Primary Schools",
    "SS": "Secondary Schools",
    "FE": "Further Education",
    "GP": "GP",
    "Hosp": "Hospitals",
    "Food": "Food Stores",
    "Town": "Town Centres",
}

def find_service(x):
    for key, value in service_names.items():
        if x[:len(key)] == key:
            return value
    return None

lsoa_journey_times["service"] = (lsoa_journey_times 
                                 .dft_variable_name 
                                 .apply(find_service))

# parse out modes of transport from dft_variable_name
mode_names = {
    "PT": "Public Transport / Walk",
    "Cyc": "Cycle",
    "Car": "Car",
    "Walk": "Walk"
}

def find_mode(x):
    for key, value in mode_names.items():
        if key in x:
            return value
    return None

lsoa_journey_times["mode_of_transport"] = (lsoa_journey_times. 
                                           dft_variable_name. 
                                           apply(find_mode))

# parse out observation types from dft_variable_name
observation_names = {
    "15n": "Origin Accessibility - Services Within 15 min",
    "30n": "Origin Accessibility - Services Within 30 min",
    "45n": "Origin Accessibility - Services Within 45 min",
    "60n": "Origin Accessibility - Services Within 60 min",
    "15pct": "Destination Accessibility - % Service Users Within 15 min",
    "30pct": "Destination Accessibility - % Service Users Within 30 min",
    "45pct": "Destination Accessibility - % Service Users Within 45 min",
    "60pct": "Destination Accessibility - % Service Users Within 60 min",
    "t": "Journey Time (min)"
}

def find_observation(x):
    for key, value in observation_names.items():
        if x[-len(key):] == key:
            return value
    return None

lsoa_journey_times["observation"] = (lsoa_journey_times 
                                  .dft_variable_name 
                                  .apply(find_observation))

index_cols = ["LSOA_code", "service", "mode_of_transport"]
lsoa_journey_times = (lsoa_journey_times
                      .pivot(index=index_cols, 
                             columns=["observation"], 
                             values=["value"])
                      .reset_index())

level_1_cols = [col for col in lsoa_journey_times.droplevel(level=1,axis=1).columns  
                if col != "value"]
level_0_cols = [col for col in lsoa_journey_times.droplevel(level=0,axis=1).columns
                if col]
lsoa_journey_times.columns = level_1_cols + level_0_cols

In [48]:
lsoa_service_users = raw_jt_data.melt(id_vars=["LSOA_code"],   
                                      value_vars=pop_cols,  
                                      var_name="service_observation",  
                                      value_name="n_service_users")

lsoa_service_users["service"] = (lsoa_service_users["service_observation"]
                                 .apply(find_service))

empl_mask = lsoa_service_users.service == "Employment Sites"
lsoa_service_users[empl_mask]

small_sites = lsoa_service_users[empl_mask].copy()
small_sites["service"] = "Employment Sites - Small"

med_sites = lsoa_service_users[empl_mask].copy()
med_sites["service"] = "Employment Sites - Medium"

lsoa_service_users.loc[empl_mask, "service"] = "Employment Sites - Large"

lsoa_service_users = (lsoa_service_users
                      .append(small_sites)
                      .append(med_sites))

lsoa_service_users.drop("service_observation", axis=1, inplace=True)

/var/folders/yn/6tpscv5n36l6nw4vrsl08qlh0000gn/T/ipykernel_1704/2698414245.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(small_sites)
/var/folders/yn/6tpscv5n36l6nw4vrsl08qlh0000gn/T/ipykernel_1704/2698414245.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(med_sites))


In [50]:
lsoa_information = raw_jt_data[lsoa_level_cols]
lsoa_information

,LSOA_code,Region,LA_Code,LA_Name,imd_rank,imd_decile,LSOA_name
0,E01000001,London,E09000001,City of London,29199,9,City of London 001A
1,E01000002,London,E09000001,City of London,30379,10,City of London 001B
2,E01000003,London,E09000001,City of London,14915,5,City of London 001C
3,E01000005,London,E09000001,City of London,8678,3,City of London 001E
4,E01000006,London,E09000002,Barking and Dagenham,14486,5,Barking and Dagenham 016A
...,...,...,...,...,...,...,...
32839,E01033764,North West,E08000012,Liverpool,116,1,Liverpool 022E
32840,E01033765,North West,E08000012,Liverpool,945,1,Liverpool 061D
32841,E01033766,North West,E08000012,Liverpool,12842,4,Liverpool 042G
32842,E01033767,North West,E08000012,Liverpool,422,1,Liverpool 050J


In [51]:
full_lsoa_jt_data = (lsoa_journey_times
                     .merge(lsoa_service_users, on=["LSOA_code", "service"], how="left")
                     .merge(lsoa_information, on="LSOA_code", how="left"))

In [52]:
full_lsoa_jt_data

,LSOA_code,service,mode_of_transport,Destination Accessibility - % Service Users Within 15 min,Destination Accessibility - % Service Users Within 30 min,Destination Accessibility - % Service Users Within 45 min,Destination Accessibility - % Service Users Within 60 min,Journey Time (min),Origin Accessibility - Services Within 15 min,Origin Accessibility - Services Within 30 min,Origin Accessibility - Services Within 45 min,Origin Accessibility - Services Within 60 min,n_service_users,Region,LA_Code,LA_Name,imd_rank,imd_decile,LSOA_name
0,E01000001,Employment Sites - Large,Car,100.000000,100.0,100.0,100.0,5.706106,10.000000,10.000000,10.000000,10.000000,1189.000000,London,E09000001,City of London,29199,9,City of London 001A
1,E01000001,Employment Sites - Large,Cycle,100.000000,100.0,100.0,100.0,5.881606,10.000000,10.000000,10.000000,10.000000,1189.000000,London,E09000001,City of London,29199,9,City of London 001A
2,E01000001,Employment Sites - Large,Public Transport / Walk,100.000000,100.0,100.0,100.0,2.518604,9.366695,10.000000,10.000000,10.000000,1189.000000,London,E09000001,City of London,29199,9,City of London 001A
3,E01000001,Employment Sites - Large,Walk,100.000000,100.0,100.0,100.0,2.518604,8.957948,10.000000,10.000000,10.000000,1189.000000,London,E09000001,City of London,29199,9,City of London 001A
4,E01000001,Employment Sites - Medium,Car,100.000000,100.0,100.0,100.0,6.039638,10.000000,10.000000,10.000000,10.000000,1189.000000,London,E09000001,City of London,29199,9,City of London 001A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313755,E01033768,Secondary Schools,Walk,14.285714,100.0,100.0,100.0,14.810000,0.285714,5.000000,9.857143,10.000000,14.000000,North West,E08000012,Liverpool,1833,1,Liverpool 037F
1313756,E01033768,Town Centres,Car,100.000000,100.0,100.0,100.0,8.646288,4.000000,10.000000,10.000000,10.000000,894.036072,North West,E08000012,Liverpool,1833,1,Liverpool 037F
1313757,E01033768,Town Centres,Cycle,100.000000,100.0,100.0,100.0,11.908180,1.262275,7.000000,10.000000,10.000000,894.036072,North West,E08000012,Liverpool,1833,1,Liverpool 037F
1313758,E01033768,Town Centres,Public Transport / Walk,0.000000,100.0,100.0,100.0,17.426359,0.000000,3.262275,10.000000,10.000000,894.036072,North West,E08000012,Liverpool,1833,1,Liverpool 037F


In [53]:
full_lsoa_jt_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1313760 entries, 0 to 1313759
Data columns (total 19 columns):
 #   Column                                                     Non-Null Count    Dtype  
---  ------                                                     --------------    -----  
 0   LSOA_code                                                  1313760 non-null  object 
 1   service                                                    1313760 non-null  object 
 2   mode_of_transport                                          1313760 non-null  object 
 3   Destination Accessibility - % Service Users Within 15 min  1313760 non-null  float64
 4   Destination Accessibility - % Service Users Within 30 min  1313760 non-null  float64
 5   Destination Accessibility - % Service Users Within 45 min  1313760 non-null  float64
 6   Destination Accessibility - % Service Users Within 60 min  1313760 non-null  float64
 7   Journey Time (min)                                         1313760 non-n

In [60]:
full_lsoa_jt_data.to_csv(data_dir + "/jt_data_long.csv", index=False)

In [56]:
raw_jt_data.shape

(32844, 375)

In [57]:
32844*375

12316500

In [58]:
full_lsoa_jt_data.shape

(1313760, 19)

In [59]:
1313760*19

24961440